Реалізація з PBKDF2

In [1]:
import os
import json
import base64
import getpass
from hashlib import pbkdf2_hmac


def derive_key(username, password):

    #Визначення кількості ітерацій:
    #Розглядаємо випадок вебаплікації, для якої важлива швидкість
    #При потребі кількість ітерацій можна збільшити
    iterations = 131072 # 2**17

    # Генерація випадкового salt (16 байтів)
    salt = os.urandom(16)

    # Обчислення ключа AES-128 (розмір 128 біт) з пароля за допомогою PBKDF2 та SHA-256
    key = pbkdf2_hmac('sha256', password.encode(), salt, iterations, dklen=16)

    # Кодування результату в специфікації PHC (зберігаємо у форматі Base64)
    phc_string = f"$pbkdf2-sha256$iter={iterations}${base64.b64encode(salt).decode()}${base64.b64encode(key).decode()}"

    # Формування метаданих у форматі JSON
    metadata = {
        "username": username,
        "phc": phc_string
    }

    # Повертаємо ключ та метадані у зручному форматі для виводу
    return key, json.dumps(metadata, indent=4)

# Приклад використання:

#Інтерактивне введення логіну та паролю (у разі потреби логін можна брати з бази)
username = input("Login: ")
password = getpass.getpass("Enter password: ")

#Обчислення 128-бітного ключа та метаданих для подального збереження
key, metadata_json = derive_key(username, password)

print("Derived AES-128 Key:", base64.b64encode(key).decode())
print("Metadata JSON:\n", metadata_json)

Login: user1
Enter password: ··········
Derived AES-128 Key: 4hHg4P3oSxzRGzYPibXQRg==
Metadata JSON:
 {
    "username": "user1",
    "phc": "$pbkdf2-sha256$iter=131072$GljYdajJUV6HRc6riHpVwg==$4hHg4P3oSxzRGzYPibXQRg=="
}


Реалізація через Argon2id

In [2]:
#Демонстрація вбудованого методу збереження PHC-рядка (без генерації ключа)
import json
from argon2 import PasswordHasher
from argon2.low_level import Type
import getpass

def derive_key_argon2id(username, password):
    ph = PasswordHasher(
        time_cost=3,
        memory_cost=65536,  # 64 MB
        parallelism=4,
        hash_len=16,        # для AES-128
        type=Type.ID        # використовуємо Argon2id
    )

    # отримання рядка з усіма необхідними параметрами згідно PHC стандарту
    phc_string = ph.hash(password)

    metadata = {
        "username": username,
        "phc": phc_string
    }

    return phc_string, json.dumps(metadata, indent=4)


# виклик функції

#Інтерактивне введення логіну та паролю (у разі потреби логін можна брати з бази)
username = input("Введіть ім'я користувача: ")
password = getpass.getpass("Введіть пароль: ")

phc_string, metadata_json = derive_key_argon2id(username, password)

print("PHC Argon2id string:\n", phc_string)
print("Metadata JSON:\n", metadata_json)


Введіть ім'я користувача: user1
Введіть пароль: ··········
PHC Argon2id string:
 $argon2id$v=19$m=65536,t=3,p=4$QzA6FI+nvTgtEaLoEzEC5w$9wlF8JPx9Wf+ffTYADqARQ
Metadata JSON:
 {
    "username": "user1",
    "phc": "$argon2id$v=19$m=65536,t=3,p=4$QzA6FI+nvTgtEaLoEzEC5w$9wlF8JPx9Wf+ffTYADqARQ"
}


In [4]:
#Генерація ключа та збереження метаданих через Argon2id
import os
import json
import base64
import getpass
from argon2.low_level import hash_secret_raw, Type

def derive_key_argon2id(username, password):

    #Визначимо параметри
    memory_cost=65536
    time_cost=3
    parallelism=4
    hash_len=16

    # Генерація випадкової солі (16 байтів)
    salt = os.urandom(16)

    # Генерація ключа Argon2id
    key = hash_secret_raw(
        secret=password.encode(),
        salt=salt,
        time_cost=time_cost,
        memory_cost=memory_cost,
        parallelism=parallelism,
        hash_len=hash_len,
        type=Type.ID #Використовуємо Argon2id
    )

    # Формування PHC-рядка
    phc_string = (f"$argon2id$v=19$m={memory_cost},t={time_cost},p={parallelism}"
                  f"${base64.b64encode(salt).decode()}${base64.b64encode(key).decode()}")

    # Метадані у форматі JSON
    metadata = {
        "username": username,
        "phc": phc_string
    }

    return key, phc_string, json.dumps(metadata, indent=4)



#Інтерактивне введення логіну та паролю (у разі потреби логін можна брати з бази)
username = input("Введіть ім'я користувача: ")
password = getpass.getpass("Введіть пароль: ")

key, phc_string, metadata_json = derive_key_argon2id(username, password)

print("Derived AES-128 Key:", base64.b64encode(key).decode())
print("PHC Argon2id string:", phc_string)
print("Metadata JSON:\n", metadata_json)


Введіть ім'я користувача: user1
Введіть пароль: ··········
Derived AES-128 Key: jXE0hLDg70Ox1zNINSUN4Q==
PHC Argon2id string: $argon2id$v=19$m=65536,t=3,p=4$Lsn8aPSybF/B6PdrS+Pctg==$jXE0hLDg70Ox1zNINSUN4Q==
Metadata JSON:
 {
    "username": "user1",
    "phc": "$argon2id$v=19$m=65536,t=3,p=4$Lsn8aPSybF/B6PdrS+Pctg==$jXE0hLDg70Ox1zNINSUN4Q=="
}
